In [1]:
import os
os.environ["http_proxy"] = "http://192.168.104.50:7890"
os.environ["https_proxy"] = "http://192.168.104.50:7890"

In [2]:
from transformers import AutoTokenizer, AutoModel
from torch import nn
import torch
import random
from d2l import torch as d2l
# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# model = AutoModel.from_pretrained("bert-base-uncased")

# inputs = tokenizer("Hello world!", return_tensors="pt")
# outputs = model(**inputs)
# outputs

In [3]:
from datasets import list_datasets, load_dataset
from pprint import pprint

# datasets = list_datasets()
# print("Number of datasets in the Datasets library: ", len(datasets), "\n\n")

# # 数据集列表
# pprint(datasets, compact=True)

In [4]:
dataset = load_dataset('sradc/chunked-wikipedia20220301en-bookcorpusopen',cache_dir='~/nvmessd/DeepLearning/datasets/',split="train")

Found cached dataset parquet (/home/ubutnu/nvmessd/DeepLearning/datasets/sradc___parquet/sradc--chunked-wikipedia20220301en-bookcorpusopen-2e9ae21627579891/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [5]:
# from transformers import BertTokenizerFast
# tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [6]:
#@save
def _get_next_sentence(sentence, next_sentence, paragraphs):
    if random.random() < 0.5:
        is_next = True
    else:
        # paragraphs是三重列表的嵌套
        next_sentence = random.choice(random.choice(paragraphs))
        is_next = False
    return sentence, next_sentence, is_next

In [7]:
#@save
def _get_nsp_data_from_paragraph(paragraph, paragraphs, vocab, max_len):
    nsp_data_from_paragraph = []
    for i in range(len(paragraph) - 1):
        tokens_a, tokens_b, is_next = _get_next_sentence(
            paragraph[i], paragraph[i + 1], paragraphs)
        # 考虑1个'<cls>'词元和2个'<sep>'词元
        if len(tokens_a) + len(tokens_b) + 3 > max_len:
            continue
        tokens, segments = d2l.get_tokens_and_segments(tokens_a, tokens_b)
        nsp_data_from_paragraph.append((tokens, segments, is_next))
    return nsp_data_from_paragraph

In [8]:
def _read_wiki(data_dir):
    file_name = os.path.join(data_dir, 'wiki.train.tokens')
    with open(file_name, 'r') as f:
        lines = f.readlines()
    # 大写字母转换为小写字母
    paragraphs = [line.strip().lower().split(' . ')
                  for line in lines if len(line.split(' . ')) >= 2]
    random.shuffle(paragraphs)
    return paragraphs

In [9]:
#@save
# def _replace_mlm_tokens(tokens, candidate_pred_positions, num_mlm_preds,
#                         vocab):
#     # 为遮蔽语言模型的输入创建新的词元副本，其中输入可能包含替换的“<mask>”或随机词元
#     mlm_input_tokens = [token for token in tokens]
#     pred_positions_and_labels = []
#     # 打乱后用于在遮蔽语言模型任务中获取15%的随机词元进行预测
#     random.shuffle(candidate_pred_positions)
#     for mlm_pred_position in candidate_pred_positions:
#         if len(pred_positions_and_labels) >= num_mlm_preds:
#             break
#         masked_token = None
#         # 80%的时间：将词替换为“<mask>”词元
#         if random.random() < 0.8:
#             masked_token = '<mask>'
#         else:
#             # 10%的时间：保持词不变
#             if random.random() < 0.5:
#                 masked_token = tokens[mlm_pred_position]
#             # 10%的时间：用随机词替换该词
#             else:
#                 masked_token = random.choice(vocab.idx_to_token)
#         mlm_input_tokens[mlm_pred_position] = masked_token
#         pred_positions_and_labels.append(
#             (mlm_pred_position, tokens[mlm_pred_position]))
#     return mlm_input_tokens, pred_positions_and_labels

In [10]:
#@save
# def _get_mlm_data_from_tokens(tokens, vocab):
#     candidate_pred_positions = []
#     # tokens是一个字符串列表
#     for i, token in enumerate(tokens):
#         # 在遮蔽语言模型任务中不会预测特殊词元
#         if token in ['<cls>', '<sep>']:
#             continue
#         candidate_pred_positions.append(i)
#     # 遮蔽语言模型任务中预测15%的随机词元
#     num_mlm_preds = max(1, round(len(tokens) * 0.15))
#     mlm_input_tokens, pred_positions_and_labels = _replace_mlm_tokens(
#         tokens, candidate_pred_positions, num_mlm_preds, vocab)
#     pred_positions_and_labels = sorted(pred_positions_and_labels,
#                                        key=lambda x: x[0])
#     pred_positions = [v[0] for v in pred_positions_and_labels]
#     mlm_pred_labels = [v[1] for v in pred_positions_and_labels]
#     return vocab[mlm_input_tokens], pred_positions, vocab[mlm_pred_labels]

In [11]:
#@save
def _pad_bert_inputs(examples, max_len, vocab):
    max_num_mlm_preds = round(max_len * 0.15)
    all_token_ids, all_segments, valid_lens,  = [], [], []
    all_pred_positions, all_mlm_weights, all_mlm_labels = [], [], []
    nsp_labels = []
    for (token_ids, pred_positions, mlm_pred_label_ids, segments,
         is_next) in examples:
        all_token_ids.append(torch.tensor(token_ids + [vocab['<pad>']] * (
            max_len - len(token_ids)), dtype=torch.long))
        all_segments.append(torch.tensor(segments + [0] * (
            max_len - len(segments)), dtype=torch.long))
        # valid_lens不包括'<pad>'的计数
        valid_lens.append(torch.tensor(len(token_ids), dtype=torch.float32))
        all_pred_positions.append(torch.tensor(pred_positions + [0] * (
            max_num_mlm_preds - len(pred_positions)), dtype=torch.long))
        # 填充词元的预测将通过乘以0权重在损失中过滤掉
        all_mlm_weights.append(
            torch.tensor([1.0] * len(mlm_pred_label_ids) + [0.0] * (
                max_num_mlm_preds - len(pred_positions)),
                dtype=torch.float32))
        all_mlm_labels.append(torch.tensor(mlm_pred_label_ids + [0] * (
            max_num_mlm_preds - len(mlm_pred_label_ids)), dtype=torch.long))
        nsp_labels.append(torch.tensor(is_next, dtype=torch.long))
    return (all_token_ids, all_segments, valid_lens, all_pred_positions,
            all_mlm_weights, all_mlm_labels, nsp_labels)

In [12]:
# paragraphs= dataset['text']

In [13]:
# paragraphs[3]

In [14]:
def replace_mask(tokens,max_len):
    tokens = tokens.split(' ')
        # 为遮蔽语言模型的输入创建新的词元副本，其中输入可能包含替换的“<mask>”或随机词元
    mlm_input_tokens = [token for token in tokens]
    pred_positions_and_labels = []
    candidate_pred_positions = [i for i in range(len(tokens))]
    num_mlm_preds = max(1, round(max_len* 0.15))
    # 打乱后用于在遮蔽语言模型任务中获取15%的随机词元进行预测
    random.shuffle(candidate_pred_positions)
    for mlm_pred_position in candidate_pred_positions:
        if len(pred_positions_and_labels) >= num_mlm_preds:
            break
        masked_token = None
        # 80%的时间：将词替换为“<mask>”词元
        if random.random() < 0.8:
            masked_token = '[MASK]'
        else:
            # 10%的时间：保持词不变
           #if random.random() < 0.5:
            masked_token = tokens[mlm_pred_position]
            # 10%的时间：用随机词替换该词
            # else:
            #     masked_token = random.choice(vocab.idx_to_token)
        mlm_input_tokens[mlm_pred_position] = masked_token
        pred_positions_and_labels.append(
            (mlm_pred_position, tokens[mlm_pred_position]))
    return mlm_input_tokens, pred_positions_and_labels

In [15]:
#@save
def get_mlm_data_from_tokens(tokens,max_len):
    # candidate_pred_positions = []
    # # tokens是一个字符串列表
    # for i, token in enumerate(tokens):
    #     # 在遮蔽语言模型任务中不会预测特殊词元
    #     if token in ['<cls>', '<sep>']:
    #         continue
    #     candidate_pred_positions.append(i)
    # 遮蔽语言模型任务中预测15%的随机词元
    mlm_input_tokens, pred_positions_and_labels = replace_mask(tokens,max_len)
    pred_positions_and_labels = sorted(pred_positions_and_labels,
                                       key=lambda x: x[0])
    pred_positions = [v[0] for v in pred_positions_and_labels]
    mlm_pred_labels = [v[1] for v in pred_positions_and_labels]
    return mlm_input_tokens, pred_positions,mlm_pred_labels

In [24]:

from tqdm.auto import tqdm
file_count=0
token = []
#front = [x.replace('\n', '').split(' ') for x in paragraphs[:10]]
# front = paragraphs[0].replace('\n', '')
for sample in tqdm(dataset['text']):
    sample =sample.replace('\n', '')
    token.append(get_mlm_data_from_tokens(sample,1024))

# dataset['text']

  0%|          | 0/33536113 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [17]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased')
#print(model)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [25]:
class Wiki_book_Dataset(torch.utils.data.Dataset): 
    def __init__(self, text,max_len): 
        self.text=text
        self.max_len=max_len
        self.max_num_mlm_preds = round(max_len * 0.15)
        #self.rand_MASk=rand_MASk()
        # if rand_MASk is None:
        #     self.text=text
        # else:
        #     self.text=rand_MASk(text)

    
    def __len__(self): 
        # return the number of samples 
        return len(self.text)
 
    def __getitem__(self, index): 
        # return dictionary of input_ids, attention_mask, and labels for index i
        #all_token_ids,all_pred_positions, all_mlm_labels=[], [],[]
        #text_mask, pred_positions,labels = get_mlm_data_from_tokens(self.text[index],self.max_len)
        # all_token_ids.append(text_mask + ['[PAD]'] * (
        #     self.max_len - len(text_mask)))
        # # text_mask=tokenizer.encode(text_mask,return_tensors="pt",padding='max_length',max_length=self.max_len)
        # all_pred_positions.append(torch.tensor(pred_positions + [0] * (self.max_num_mlm_preds - len(pred_positions)), dtype=torch.long))
        # all_mlm_labels.append((labels + [0] * (
        #     self.max_num_mlm_preds - len(labels))))
        # labels=tokenizer.encode(labels,return_tensors="pt",padding='max_length',max_length=self.max_num_mlm_preds)
        return self.text[index]
    # candidate_pred_positions = []
    # # tokens是一个字符串列表
    # for i, token in enumerate(tokens):
    #     # 在遮蔽语言模型任务中不会预测特殊词元
    #     if token in ['<cls>', '<sep>']:
    #         continue
    #     candidate_pred_positions.append(i)
    # 遮蔽语言模型任务中预测15%的随机词元


In [26]:
max_len=512
train_dataset=Wiki_book_Dataset(dataset['text'],max_len)
# train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=128,num_workers=8,shuffle=True)
# inputs = tokenizer(toke[0],return_tensors="pt")
# model.train()
# output = model(**inputs)
# #example_text = tokenizer.decode(inputs.input_ids[0])
# #tokenizer.get_special_tokens_mask(inputs.tokens_id_0)
# inputs,output

In [20]:
# text_mask, pred_positions,labels = get_mlm_data_from_tokens(dataset['text'][0],256)

In [21]:
# type(text_mask)


In [29]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=64,num_workers=4,shuffle=False)

In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
for i,X in enumerate(train_loader):
    if i >1:
        break
    #print(X[0])
    print(tokenizer.batch_encode_plus(X,return_tensors="pt",padding='max_length',max_length=512,truncation=True))


In [ ]:
from torch.optim import Adam
from tqdm import tqdm

def train(model, train_data, val_data, learning_rate, epochs,max_len):
  # 通过Dataset类获取训练和验证集
    #train, val = Dataset(train_data), Dataset(val_data)
    train_dataset=Wiki_book_Dataset(train_data,max_len)
    # DataLoader根据batch_size获取数据，训练时选择打乱样本
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=64,num_workers=4,shuffle=False)
  # 判断是否使用GPU
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    # 定义损失函数和优化器
    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=learning_rate)

    if use_cuda:
            model = model.cuda()
            criterion = criterion.cuda()
    # 开始进入训练循环
    for epoch_num in range(epochs):
      # 定义两个变量，用于存储训练集的准确率和损失
            total_acc_train = 0
            total_loss_train = 0
      # 进度条函数tqdm
            for train_input, train_label in tqdm(train_loader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)
        # 通过模型得到输出
                output = model(input_id, mask)
                # 计算损失
                batch_loss = criterion(output, train_label)
                total_loss_train += batch_loss.item()
                # 计算精度
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc
        # 模型更新
                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            # ------ 验证模型 -----------
            # 定义两个变量，用于存储验证集的准确率和损失
            total_acc_val = 0
            total_loss_val = 0
      # 不需要计算梯度
            with torch.no_grad():
                # 循环获取数据集，并用训练好的模型进行验证
                for val_input, val_label in val_dataloader:
          # 如果有GPU，则使用GPU，接下来的操作同训练
                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)
  
                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label)
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'''Epochs: {epoch_num + 1} 
              | Train Loss: {total_loss_train / len(train_data): .3f} 
              | Train Accuracy: {total_acc_train / len(train_data): .3f} 
              | Val Loss: {total_loss_val / len(val_data): .3f} 
              | Val Accuracy: {total_acc_val / len(val_data): .3f}''')  